In [1]:
#from gradio.layouts import Column
#import gradio as gr
from pandas.io.parquet import read_parquet
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD

In [20]:
dfrating = read_parquet(r'./total_plataformas1.parquet', columns=['userId', 'show_id','rating_y'])
dfrating.head()

,userId,show_id,rating_y
0,543,as1,5.0
1,595,as1,3.0
2,611,as1,3.0
3,2523,as1,3.5
4,3082,as1,4.0


In [10]:
from sklearn.preprocessing import LabelEncoder
col = dfrating.show_id 

# creamos objeto labelencoder y ajustamos la columna
labelE = LabelEncoder()
labelE.fit(col)

#tranformamos la columna
col_transformada = labelE.transform(col)

#lo reemplazamos a show id original
dfrating.show_id = col_transformada
dfrating.head(3)

,userId,show_id,rating_y
9844085,113699,20132,3.5
1938897,63058,3383,0.5
4329778,60198,8930,1.0


In [21]:
dfrating.duplicated().sum()

28655

In [12]:
dfrating1 = dfrating.drop_duplicates()

In [13]:
dfrating1.duplicated().sum()

0

In [22]:
dftitle = read_parquet(r'./total_plataformas1.parquet', columns=['show_id', 'title'])
dftitle.head()

,show_id,title
0,as1,the grand seduction
1,as1,the grand seduction
2,as1,the grand seduction
3,as1,the grand seduction
4,as1,the grand seduction


### Definimos el intervalo de rating

In [23]:
reader = Reader(rating_scale=(1, 5))

In [24]:
# pasamos de un dataframe a formato surprise lo cual debe tener un orden user, item, rating
data = Dataset.load_from_df(dfrating1[['userId','show_id','rating_y']], reader)

# Pasos Machine Learning
- Dividir la data en train y test
- instanciar
- Entrenamos
- Predecimos
- Evaluar

In [25]:
#dividimos el dato en train y test de 0.3
trainset, testset = train_test_split(data, test_size=0.3)

#instanciamos
model = SVD()

#entrenamos el modelo 
model.fit(trainset)

#predecimos
predictions = model.test(testset)

In [30]:
#prediccion
userid = 1
movieid = 'as1'

#ver si la pelicula esta en mi base de datos
if movieid not in dftitle.show_id.values:
    print('La pelicula con el Id', userid, 'no existe')

#ver si mi usuario esta en mi data
if userid not in dfrating.userId.values:
    print('El usuario con id', userid, 'no existe')

# obtenemos calificacion
user_rating = dfrating[dfrating.userId == userid]

# peliculas vistas por el usuario
user_movies = list(user_rating['show_id'])

if movieid in user_movies:
    print('el usuario ya ha visto la peliculas')
else:
    prediction = model.predict(userid, movieid)
    print('El puntaje la pelicula con el id',movieid,'del usuario con el id',userid,'es', prediction.est)
    if prediction.est >= 3.5:
        print('recomiendo')
    else:
        print('no recomendada')

El puntaje la pelicula con el id as1 del usuario con el id 1 es 4.097989814622066
recomiendo


## Evaluacion del modelo

In [33]:
from surprise import accuracy

a = accuracy.rmse(predictions)

print('Al ser un numero bajo el RMSE:',a, 'es un buen modelo')

RMSE: 1.0023
Al ser un numero bajo el RMSE: 1.0022587169527448 es un buen modelo
